In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline 

In [ ]:
# a.u. are used
def psi_b(r, t): # r is supposed to be a vector [x,y,z,m]
    beta = 1
    Eb = - 0.5
    psi = np.sqrt(beta**(1/3))/np.pi * \
          np.multiply(
              np.exp(-beta*r).reshape(r.shape[0],1), \
              np.exp(-1j*Eb*t).reshape(1,t.shape[0]))
    #print(np.exp(-beta*r).shape*np.exp(-1j*Eb*t).shape)
    return psi

In [ ]:
x = np.linspace(0,10,128)
t = np.linspace(-10,10,128)
print(psi_b(x,t).shape)
#plt.plot(x,np.abs(psi_b(x,t)))
#plt.plot(x, np.real(psi_b(x,t)))
#plt.plot(x, np.imag(psi_b(x,t)))

In [ ]:
# not ready yet
def psi_b_plus(r,t):
    Ep = 1 # ???
    u_plus = 1 # ???
    c = 0 #integration constant
    a_0 = - e * A0 / wL # A0 should be a vectror
    a_L = a_0*sin(wL*t) # or maybe it is worth to make it a function 
    
    psi = np.exp(
        -1j*Ep*t - \
        i*alpha_L(t)*p - \
        i/2/m * eL**2 * A0**2 *(t + sin(2*wL*t))/2 \
        + c) * \
    u_plus(r)
    return psi

In [502]:
#from scipy.special import hyp1f1
from mpmath import hyp1f1, gamma
from mpmath import exp as mp_exp 
from mpmath import matrix as mp_matrix

In [503]:
#p = [1,2,-3j]
#a = np.linalg.norm(p,2)
def hyper_1F1(r,p): #input just 2 vectors not matrix 
    """it is supposed to return float32 type, but returns mpf"""
    
    Z = 1
    h_bar = 1
    a0 = 1
    eta_p = Z*h_bar/a0/np.linalg.norm(p,2, axis = 0) # is abs equal to norm?
    #print(np.linalg.norm(p,2, axis = 0),p,p.shape,type(p))
    
    hyp1f1_fun = np.frompyfunc(hyp1f1, 3, 1) # vector, USAGE IS NOT CORRECT
    #result = hyp1f1_fun(1, 1, np.array(z)) 
    hfun = hyp1f1_fun(
        -1j*eta_p,\
        1,\
        -1j*(np.linalg.norm(p,2, axis = 0)*np.linalg.norm(r,2, axis = 0) + np.dot(p,r))/h_bar
    )
    #print(type(hfun))
    return hfun

In [504]:
#Test
r = np.array([1, 2, 3])
p = np.array([4, 5, 6])
print(hyper_1F1(r,p))

(0.723236263122129 + 0.434200878681639j)


In [505]:
def matrix_hyper_1F1_p(r,p): # we have fixed p here, and go over r vectors
    
    """think about loop removing"""
    #hyp1f1_fun = np.frompyfunc(hyper_1F1, 2, 1)
    #hfun = hyp1f1_fun(r, p)
    values = mp_matrix(1,len(r))
    
    for i in range(len(r)):
        hfun = hyper_1F1(r[i],p)
        values[0,i] = hfun
    return values

In [506]:
#Test
r = np.array([[1, 2, 3], [7, 8, 9]])
p = np.array([7, 8, 9])
print(matrix_hyper_1F1_p(r,p), p.shape)

[(0.835061926636437 + 0.326741903074577j)  (0.800228898980005 + 0.405875072282826j)] (3,)


In [507]:
"""hyp1f1_fun = np.frompyfunc(hyp1f1, 3, 1) # make vector input function 
values = hyp1f1_fun(1, 1, np.array(z)) 
Z = np.array(values.tolist(), dtype = np.float32) # convert mpf type to float32
print(Z, type(Z))"""


'hyp1f1_fun = np.frompyfunc(hyp1f1, 3, 1) # make vector input function \nvalues = hyp1f1_fun(1, 1, np.array(z)) \nZ = np.array(values.tolist(), dtype = np.float32) # convert mpf type to float32\nprint(Z, type(Z))'

In [508]:
def matrix_hyp1F1_px(r,p): # p is not fixed anymore
    """think about loop removing"""
    #hyp1f1_fun = np.frompyfunc(hyper_1F1, 2, 1)
    #hfun = hyp1f1_fun(r, p)
    values = mp_matrix(len(p), len(r))
    
    for i in range(len(p)):
        hfun = matrix_hyper_1F1_p(r,p[i])
        values[i,:] = hfun
    return values

    # rows - p changes
    # columns - r changes

In [509]:
#Test
r = np.array([[1, 2, 3], [7, 8, 9]])
p = np.array([[4, 5, 6], [7, 8, 9]])
print(matrix_hyp1F1_px(r,p), p.shape)

[(0.723236263122129 + 0.434200878681639j)  (0.651149846811972 + 0.539129042554001j)]
[(0.835061926636437 + 0.326741903074577j)  (0.800228898980005 + 0.405875072282826j)] (2, 3)


In [510]:
def my_gamma(p):
    Z = 1
    h_bar = 1
    a0 = 1
    
    my_gamma = np.frompyfunc(gamma, 1, 1) # vector
    eta_p = Z*h_bar/a0/np.linalg.norm(p,2, axis = 1) # why does the other axis is used in hyperfunction?
    values = my_gamma(1 + 1j*eta_p).tolist()
    print(type(values))
    return values

In [511]:
def inner_gamma(p):
    Z = 1
    h_bar = 1
    a0 = 1
    
    #my_gamma = np.frompyfunc(gamma, 1, 1) # vector
    eta_p = Z*h_bar/a0/np.linalg.norm(p,2, axis = 0)
    gamma(1 + 1j*eta_p)
    
    return gamma(1 + 1j*eta_p)

def my_gamma(p):
    values = mp_matrix(1,len(p))
    
    for i in range(len(p)):
        values[0,i] = inner_gamma(p[i])
    return values

In [512]:
#Test
p = np.array([[4, 5, 6], [5, 6, 6]])
print(my_gamma(p), type(my_gamma(p)))

[(0.987318550276245 - 0.0644553834665771j)  (0.989906808681611 - 0.0576679559640922j)] <class 'mpmath.matrices.matrices.matrix'>


In [537]:
from mpmath import mpc

def columb_wave_single(r, p): # Maybe some functions above are redundant
    
    h_bar = 1
    Z = 1
    a0 = 1
    L = 3
    
    eta_p = Z*h_bar/a0/np.linalg.norm(p,2, axis = 0)
    
    #phi = 
    phi = np.exp(eta_p*np.pi/2)*\
        inner_gamma(p)*\
        mpc(np.exp(1j*np.dot(p,r)/h_bar))*\
        hyper_1F1(r,p) # * (1/L)**(3/2)
    
    #how to normilize this function?
    return phi

In [538]:
r = np.array([1, 2, 3])
p = np.array([4, 5, 6])
print(columb_wave_single(r, p))

(0.488397421999974 + 0.870621463643977j)


In [536]:
"""
def matrix_columb_wave_p(r,p): # we have fixed p here, and go over r vectors
    
    """think about loop removing"""
    #hyp1f1_fun = np.frompyfunc(hyper_1F1, 2, 1)
    #hfun = hyp1f1_fun(r, p)
    values = mp_matrix(1,len(r))
    
    for i in range(len(r)):
        hfun = columb_wave_single(r[i],p)
        values[0,i] = hfun
    return values

p = np.array([1, 2, 3])
r = np.array([[4, 5, 6], [5, 6, 7]])
print(matrix_columb_wave_p(r,p))
"""

SyntaxError: invalid syntax (<ipython-input-536-dc33acbb8cc9>, line 4)

In [619]:
def col_vol_state(r, p, t):
    c = 0
    m = 1
    wL = 1
    A0 = np.array([0, 1, 0])
    e = 1
    Ep =2
    alpha_0 = - e * A0/wL 
    
    alpha_L = alpha_0 * np.sin(wL * t) # should be fun
    
    integ_vec = (np.linalg.norm(A0,2) * e**2)*(1/2) * \
    (t + np.sin(2*wL*t))/2/wL + c # should be fun
    
    state = np.exp(-1j*Ep*t - \
                   1j*alpha_L*p - \
                  -1j*integ_vec/2/m)
    
      
    c_wave = columb_wave_single(r, p)
    np_c_wave = float(c_wave.real) + 1j*float(c_wave.imag) # np?
   
    #print(state, type(state), np_c_wave)
    state = state*np_c_wave
    
    #print(state, type(state))
    
    return state

In [625]:
r = np.array([1, 2, 3])
p = np.array([4, 5, 6])
t = 1
print(col_vol_state(r,p,t))

[ 0.76236691-0.64444586j -0.93285345-0.35538459j  0.76236691-0.64444586j]


In [587]:
A = np.array([1,2,3])
np.linalg.norm(A,2)

3.7416573867739413

In [596]:
mpf([1j,2j])

NameError: name 'mpf' is not defined